In [1]:
import pandas as pd
import glob
import re

# Read all CSVs that start with imdb_ and end with .csv
files = glob.glob("imdb_*.csv")

# Merge into one DataFrame
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

# Save combined file
df.to_csv("imdb_movies", index=False)

print(f"✅ Merged {len(files)} files, total {len(df)} movies saved to imdb_all_genres_2024.csv")


✅ Merged 6 files, total 7972 movies saved to imdb_all_genres_2024.csv


In [2]:
df_unique_names = df.drop_duplicates(subset=['title'])

In [3]:
print(df_unique_names)


                                                title duration  rating voters  \
0                               1. Mahavatar Narsimha   2h 10m     9.2    32K   
1                                            2. Relay   1h 52m     7.3   1.3K   
2                                     3. Freaky Tales   1h 47m     6.3    11K   
3                                   4. Dune: Part Two   2h 46m     8.5   660K   
4                                     5. Gladiator II   2h 28m     6.5   270K   
...                                               ...      ...     ...    ...   
7967                        2245. Mind and Distortion      57m     NaN    NaN   
7968  2246. Yean Yeathuku Yeppadi from Dishoreabinash      49m     NaN    NaN   
7969                                  2247. The Birth      NaN     NaN    NaN   
7970                                  2248. Seclusion      18m     NaN    NaN   
7971                            2249. Drive All Night      NaN     NaN    NaN   

         genre  
0       Ac

In [4]:
df.dtypes

title        object
duration     object
rating      float64
voters       object
genre        object
dtype: object

In [5]:
print("🔍 Duplicate Count:", df.duplicated().sum())
df = df.drop_duplicates()
print("✅ After removing duplicates:", df.shape)

🔍 Duplicate Count: 0
✅ After removing duplicates: (7972, 5)


In [6]:
if "title" in df.columns:
    # Extract number from title if present
    df["sl_no"] = df["title"].str.extract(r"^(\d+)")
    df["title"] = df["title"].str.replace(r"^\d+\.\s*", "", regex=True)

    # Convert to integer (nullable int first)
    df["sl_no"] = df["sl_no"].astype(pd.Int64Dtype())

    # Auto-generate if missing
df["sl_no"] = df["sl_no"].fillna(pd.Series(df.index + 1, index=df.index)).astype(int)

In [34]:
df

,id,title,duration,rating,voters,genre
0,1,Mahavatar Narsimha,130.0,9.2,32000.0,Action
1,2,Relay,112.0,7.3,13000.0,Action
2,3,Freaky Tales,107.0,6.3,11000.0,Action
3,4,Dune: Part Two,166.0,8.5,660000.0,Action
4,5,Gladiator II,148.0,6.5,270000.0,Action
...,...,...,...,...,...,...
7963,7964,Preet Binjara,116.0,5.9,138.0,Thriller
7964,7965,The Dogs Separation,90.0,5.9,138.0,Thriller
7965,7966,Contro un iceberg di polistirolo,120.0,5.9,138.0,Thriller
7969,7970,The Birth,95.0,5.9,138.0,Thriller


In [8]:
# ---------------- STEP 5: Convert Duration → Minutes ----------------
def convert_duration(x):
    if pd.isna(x): 
        return None
    hours = re.search(r"(\d+)h", str(x))
    mins = re.search(r"(\d+)m", str(x))
    total = 0
    if hours:
        total += int(hours.group(1)) * 60
    if mins:
        total += int(mins.group(1))
    return total if total > 0 else None

if "duration" in df.columns:
    df["duration"] = df["duration"].apply(convert_duration)


In [33]:
df

,id,title,duration,rating,voters,genre
0,1,Mahavatar Narsimha,130.0,9.2,32000.0,Action
1,2,Relay,112.0,7.3,13000.0,Action
2,3,Freaky Tales,107.0,6.3,11000.0,Action
3,4,Dune: Part Two,166.0,8.5,660000.0,Action
4,5,Gladiator II,148.0,6.5,270000.0,Action
...,...,...,...,...,...,...
7963,7964,Preet Binjara,116.0,5.9,138.0,Thriller
7964,7965,The Dogs Separation,90.0,5.9,138.0,Thriller
7965,7966,Contro un iceberg di polistirolo,120.0,5.9,138.0,Thriller
7969,7970,The Birth,95.0,5.9,138.0,Thriller


In [31]:
# Count movies with duration < 60
short_count = df[df['duration'] < 60].shape[0]

print(f"Number of movies with duration below 60 minutes: {short_count}")



Number of movies with duration below 60 minutes: 357


In [35]:
# Drop movies with duration < 60
df = df[df['duration'] >= 60]

print(f"After dropping, remaining movies: {df.shape[0]}")


After dropping, remaining movies: 7615


In [36]:
# ---------------- STEP 6: Convert Voters → Numeric ----------------
def convert_voters(x):
    if pd.isna(x):
        return None
    # Clean input: lowercase, remove dots & extra spaces
    x = str(x).lower().replace(".", "").strip()
    
    if "k" in x:
        return float(x.replace("k", "")) * 1000
    elif "m" in x:
        return float(x.replace("m", "")) * 1000000
    else:
        try:
            return float(x)
        except:
            return None

if "voters" in df.columns:
    df["voters"] = df["voters"].apply(convert_voters)
    df["voters"] = df["voters"].fillna(df["voters"].median())

In [37]:
df

,id,title,duration,rating,voters,genre
0,1,Mahavatar Narsimha,130.0,9.2,320000.0,Action
1,2,Relay,112.0,7.3,130000.0,Action
2,3,Freaky Tales,107.0,6.3,110000.0,Action
3,4,Dune: Part Two,166.0,8.5,6600000.0,Action
4,5,Gladiator II,148.0,6.5,2700000.0,Action
...,...,...,...,...,...,...
7963,7964,Preet Binjara,116.0,5.9,1380.0,Thriller
7964,7965,The Dogs Separation,90.0,5.9,1380.0,Thriller
7965,7966,Contro un iceberg di polistirolo,120.0,5.9,1380.0,Thriller
7969,7970,The Birth,95.0,5.9,1380.0,Thriller


In [38]:
df.dtypes

id            int64
title        object
duration    float64
rating      float64
voters      float64
genre        object
dtype: object

In [39]:
df.isna().sum()

id          0
title       0
duration    0
rating      0
voters      0
genre       0
dtype: int64

In [40]:
# ---------------- STEP 3: Handle Nulls (Median) ----------------
df.fillna(
    {
        "rating": df["rating"].median(),
        "voters": df["voters"].median(),
        "duration": df["duration"].median()
    },
    inplace=True
)



In [57]:
#✅ See which titles are duplicated (with counts)
duplicates = df['title'].value_counts()
duplicates = duplicates[duplicates > 1]
duplicate_count = df.duplicated(subset=['title']).sum()
print(f"Total duplicate titles: {duplicate_count}")

print("Duplicate movie titles with their counts:")
print(duplicates)


Total duplicate titles: 1819
Duplicate movie titles with their counts:
title
Escape                            6
The Killer                        5
Dr. Jason II: Lights & Shadows    5
Karma                             5
The Epidemic                      5
                                 ..
Lumina                            2
Mirza: Part 1 - Joker             2
Yolki 11                          2
Tipppsy                           2
Possession: Kerasukan             2
Name: count, Length: 1438, dtype: int64


In [58]:
# Count total duplicates
duplicate_count = df.duplicated(subset=['title']).sum()
print(f"Total duplicate titles: {duplicate_count}")


Total duplicate titles: 1819


In [59]:
#keep the highest-rated version instead of the first
df = df.sort_values('rating', ascending=False).drop_duplicates(subset=['title'], keep='first')


In [60]:
df

,id,title,duration,rating,voters,genre
4819,5048,Mi Ilustrísimo Amigo,80.0,9.8,60.0,Romance
7021,7320,Dheera Samrat,123.0,9.7,1000.0,Thriller
6840,7135,Faction Leni Seemakatha,95.0,9.7,540.0,Thriller
3939,4131,Tears of Blood,88.0,9.7,80.0,Mystery
6505,6790,31st Movie,145.0,9.6,190.0,Thriller
...,...,...,...,...,...,...
2235,2325,Amityville: Where the Echo Lives,89.0,1.5,4740.0,Horror
790,805,Daanvi,117.0,1.5,90.0,Action
6690,6978,Q: Deep Fake,60.0,1.4,140.0,Thriller
2821,2928,Cin Köyü,75.0,1.0,150.0,Horror


In [69]:
df.dtypes

id            int64
title        object
duration    float64
rating      float64
voters      float64
genre        object
dtype: object

In [70]:
# ---------------- STEP 8: Add Auto ID + Reorder ----------------
df.reset_index(drop=True, inplace=True)
df.insert(0, "id", df.index + 1)   # add id column at front

desired_order = ["id", "title", "duration", "rating", "voters", "genre"]
df = df[[col for col in desired_order if col in df.columns]]

ValueError: cannot insert id, already exists

In [62]:
# Drop sl_no if you don't need it
if "sl_no" in df.columns:
    df.drop(columns=["sl_no"], inplace=True)

In [71]:
df

,id,title,duration,rating,voters,genre
0,5048,Mi Ilustrísimo Amigo,80.0,9.8,60.0,Romance
1,7320,Dheera Samrat,123.0,9.7,1000.0,Thriller
2,7135,Faction Leni Seemakatha,95.0,9.7,540.0,Thriller
3,4131,Tears of Blood,88.0,9.7,80.0,Mystery
4,6790,31st Movie,145.0,9.6,190.0,Thriller
...,...,...,...,...,...,...
5791,2325,Amityville: Where the Echo Lives,89.0,1.5,4740.0,Horror
5792,805,Daanvi,117.0,1.5,90.0,Action
5793,6978,Q: Deep Fake,60.0,1.4,140.0,Thriller
5794,2928,Cin Köyü,75.0,1.0,150.0,Horror


In [64]:
!pip install sqlalchemy
!pip install tabulate
!pip install pandas


In [73]:
import pymysql

# --- TiDB Connection Details ---
user = "2JRRhPHCS6mRsGW.root"
password = "mt2YyedoEbMz4suV"
host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com"
port = 4000

# Step 1: Connect without selecting a database
conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    port=port,
    ssl={"ssl_verify_cert": True, "ssl_verify_identity": True}
)

cur = conn.cursor()
cur.execute("CREATE DATABASE IF NOT EXISTS imdb;")
print("✅ Database `imdb` created (or already exists).")

cur.close()
conn.close()


✅ Database `imdb` created (or already exists).


In [74]:
from sqlalchemy import create_engine
import pandas as pd

database = "imdb"

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}",
    connect_args={
        "ssl": {
            "ssl_verify_cert": True,
            "ssl_verify_identity": True
        }
    }
)

# Save DataFrame to TiDB
df.to_sql(
    "imdb_movies",
    con=engine,
    if_exists="replace",
    index=False
)

print("🎉 DataFrame stored in TiDB -> imdb.imdb_movies")


🎉 DataFrame stored in TiDB -> imdb.imdb_movies


In [75]:
df.to_sql("imdb_movies", con=engine, index=False, if_exists="replace")
df.to_csv("imdb_movies", index=False)
